# Doing some Imports

In [1]:
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D,Input, Add, Dense, ZeroPadding2D, BatchNormalization, Flatten, AveragePooling2D,GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
import cv2

## Create your model

In [2]:
def create_model(input_shape,Filters):
    # TODO: Create the Model
    F1,F2,F3,F4 = Filters
    model = Sequential()
    #Batch normalization
    model.add(BatchNormalization(input_shape=input_shape))
    #1st convolution
    model.add(Conv2D(filters = F1,kernel_size = (3, 3),strides = (1, 1),padding = 'valid',activation = 'relu'))
    #2nd convolution
    model.add(Conv2D(filters = F2,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    #Max pooling
    model.add(MaxPooling2D(pool_size =(2, 2)))
    #3 convolutions that include dropout
    model.add(Conv2D(filters = F2,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters = F2,kernel_size = (3, 3),strides = (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
   
    model.add(Conv2D(filters = F3,kernel_size = (3, 3),strides = (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    #Max pooling
    model.add(MaxPooling2D(pool_size =(2, 2)))
    #2 convs including dropout
    model.add(Conv2D(filters = F4,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(filters = F4,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    #Max pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    #Upsampling
    model.add(UpSampling2D(size=(2, 2)))
    
    #2 deconvolution
    model.add(Conv2DTranspose(filters = F4,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2DTranspose(filters = F4,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    #Upsampling
    model.add(UpSampling2D(size=(2, 2)))
    
    #3 deconvs
    model.add(Conv2DTranspose(filters = F3,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2DTranspose(filters = F3,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    model.add(Conv2DTranspose(filters = F2,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    model.add(Dropout(0.2))
    
    #Upsampling
    model.add(UpSampling2D(size=(2, 2)))
    #2 deconvs
    model.add(Conv2DTranspose(filters = F2,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    
    model.add(Conv2DTranspose(filters = 1,kernel_size = (3, 3),strides =  (1, 1),padding = 'valid',activation = 'relu'))
    
    return model

## Load the data

In [3]:
# Load training images
train_images = pickle.load(open("C:/Users/vip/Documents/CESS/deep learning/labs/project/Baseline/full_CNN_train.p", "rb" ))

# Load image labels
labels = pickle.load(open("C:/Users/vip/Documents/CESS/deep learning/labs/project/Baseline/full_CNN_labels.p", "rb" ))

# change into arrays
train_images = np.array(train_images)
labels = np.array(labels) 


## Preprocess the data

In [4]:
# TODO: Preprocess the images
from PIL import Image
from skimage import color
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
#Normalization
labels = labels / 255

## Create the train and test splits

In [5]:
# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)


## Train your model

In [6]:
# TODO: Train your model
batch_size = 64
epochs = 10
input_shape = X_train.shape[1:]

batch_size1 = 128
epochs1 = 15
# Create the neural network
model = create_model(input_shape,[2,4,8,16])
model1 = create_model(input_shape,[2,4,8,16])
model2 = create_model(input_shape,[2,4,8,16])
model3 = create_model(input_shape,[2,4,8,16])
model4 = create_model(input_shape,[2,4,8,16])
model5 = create_model(input_shape,[2,4,8,16])
datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)

In [7]:
from tensorflow import keras
optimizer = keras.optimizers.Adam(learning_rate=0.001)
optimizer1 = keras.optimizers.SGD(learning_rate=0.01,momentum= 0.5)
optimizer2 = keras.optimizers.Adam(learning_rate = 1e-3)

In [11]:
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics = ['accuracy']) 
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1,validation_data=(X_val, y_val))

C:\Users\alaam\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
179/179 [==============================] - 198s 1s/step - loss: 0.0606 - accuracy: 0.9016 - val_loss: 0.0532 - val_accuracy: 0.8731
Epoch 2/10
179/179 [==============================] - 190s 1s/step - loss: 0.0230 - accuracy: 0.9464 - val_loss: 0.0241 - val_accuracy: 0.9466
Epoch 3/10
179/179 [==============================] - 194s 1s/step - loss: 0.0183 - accuracy: 0.9504 - val_loss: 0.0189 - val_accuracy: 0.9515
Epoch 4/10
179/179 [==============================] - 197s 1s/step - loss: 0.0159 - accuracy: 0.9519 - val_loss: 0.0161 - val_accuracy: 0.9528
Epoch 5/10
179/179 [==============================] - 199s 1s/step - loss: 0.0143 - accuracy: 0.9530 - val_loss: 0.0138 - val_accuracy: 0.9548
Epoch 6/10
179/179 [==============================] - 235s 1s/step - loss: 0.0134 - accuracy: 0.9538 - val_loss: 0.0150 - val_accuracy: 0.9541
Epoch 7/10
179/179 [==============================] - 213s 1s/step - loss: 0.0127 - accuracy: 0.9543 - val_loss: 0.0124 - val_accuracy: 0.9556

In [12]:
model1.compile(optimizer=optimizer1, loss='mean_squared_error', metrics = ['accuracy']) 
model1.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1, validation_data=(X_val, y_val))

Epoch 1/10
179/179 [==============================] - 774s 4s/step - loss: 0.1284 - accuracy: 0.8289 - val_loss: 0.1202 - val_accuracy: 0.8301
Epoch 2/10
179/179 [==============================] - 490s 3s/step - loss: 0.1100 - accuracy: 0.8289 - val_loss: 0.0918 - val_accuracy: 0.8301
Epoch 3/10
179/179 [==============================] - 246s 1s/step - loss: 0.0904 - accuracy: 0.8361 - val_loss: 0.0786 - val_accuracy: 0.8584
Epoch 4/10
179/179 [==============================] - 193s 1s/step - loss: 0.0806 - accuracy: 0.8523 - val_loss: 0.0723 - val_accuracy: 0.8724
Epoch 5/10
179/179 [==============================] - 189s 1s/step - loss: 0.0746 - accuracy: 0.8649 - val_loss: 0.0682 - val_accuracy: 0.8835
Epoch 6/10
179/179 [==============================] - 194s 1s/step - loss: 0.0701 - accuracy: 0.8745 - val_loss: 0.0646 - val_accuracy: 0.8890
Epoch 7/10
179/179 [==============================] - 192s 1s/step - loss: 0.0663 - accuracy: 0.8826 - val_loss: 0.0605 - val_accuracy: 0.8947

In [19]:
model2.compile(optimizer=optimizer2, loss='mean_squared_error', metrics = ['accuracy']) 
model2.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1, validation_data=(X_val, y_val))

Epoch 1/10
179/179 [==============================] - 593s 3s/step - loss: 0.0520 - accuracy: 0.9052 - val_loss: 0.0547 - val_accuracy: 0.9086
Epoch 2/10
179/179 [==============================] - 114s 634ms/step - loss: 0.0519 - accuracy: 0.9055 - val_loss: 0.0547 - val_accuracy: 0.9086
Epoch 3/10
179/179 [==============================] - 90s 501ms/step - loss: 0.0519 - accuracy: 0.9054 - val_loss: 0.0547 - val_accuracy: 0.9086
Epoch 4/10
179/179 [==============================] - 83s 460ms/step - loss: 0.0520 - accuracy: 0.9052 - val_loss: 0.0547 - val_accuracy: 0.9086
Epoch 5/10
179/179 [==============================] - 82s 455ms/step - loss: 0.0519 - accuracy: 0.9052 - val_loss: 0.0547 - val_accuracy: 0.9086
Epoch 6/10
179/179 [==============================] - 83s 464ms/step - loss: 0.0519 - accuracy: 0.9053 - val_loss: 0.0547 - val_accuracy: 0.9086
Epoch 7/10
179/179 [==============================] - 86s 477ms/step - loss: 0.0519 - accuracy: 0.9053 - val_loss: 0.0547 - val_acc

In [14]:
model3.compile(optimizer=optimizer, loss='mean_squared_error', metrics = ['accuracy']) 
model3.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size1), steps_per_epoch=len(X_train)/batch_size1,
    epochs=epochs1, verbose=1, validation_data=(X_val, y_val))

Epoch 1/15
89/89 [==============================] - 806s 9s/step - loss: 0.0533 - accuracy: 0.9067 - val_loss: 0.0257 - val_accuracy: 0.9428
Epoch 2/15
89/89 [==============================] - 185s 2s/step - loss: 0.0208 - accuracy: 0.9478 - val_loss: 0.0163 - val_accuracy: 0.9530
Epoch 3/15
89/89 [==============================] - 181s 2s/step - loss: 0.0168 - accuracy: 0.9511 - val_loss: 0.0140 - val_accuracy: 0.9531
Epoch 4/15
89/89 [==============================] - 180s 2s/step - loss: 0.0152 - accuracy: 0.9520 - val_loss: 0.0125 - val_accuracy: 0.9542
Epoch 5/15
89/89 [==============================] - 180s 2s/step - loss: 0.0143 - accuracy: 0.9526 - val_loss: 0.0117 - val_accuracy: 0.9548
Epoch 6/15
89/89 [==============================] - 183s 2s/step - loss: 0.0135 - accuracy: 0.9531 - val_loss: 0.0112 - val_accuracy: 0.9553
Epoch 7/15
89/89 [==============================] - 204s 2s/step - loss: 0.0129 - accuracy: 0.9535 - val_loss: 0.0107 - val_accuracy: 0.9556
Epoch 8/15
89

In [8]:
model4.compile(optimizer='Adam', loss='mean_squared_error', metrics = ['accuracy']) 
model4.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1,validation_data=(X_val, y_val))

C:\Users\vip\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
179/179 [==============================] - 527s 3s/step - loss: 0.0578 - accuracy: 0.9020 - val_loss: 0.0316 - val_accuracy: 0.9409
Epoch 2/10
179/179 [==============================] - 243s 1s/step - loss: 0.0216 - accuracy: 0.9471 - val_loss: 0.0236 - val_accuracy: 0.9447
Epoch 3/10
179/179 [==============================] - 250s 1s/step - loss: 0.0171 - accuracy: 0.9504 - val_loss: 0.0182 - val_accuracy: 0.9488
Epoch 4/10
179/179 [==============================] - 238s 1s/step - loss: 0.0152 - accuracy: 0.9515 - val_loss: 0.0156 - val_accuracy: 0.9508
Epoch 5/10
179/179 [==============================] - 224s 1s/step - loss: 0.0140 - accuracy: 0.9524 - val_loss: 0.0119 - val_accuracy: 0.9542
Epoch 6/10
179/179 [==============================] - 224s 1s/step - loss: 0.0130 - accuracy: 0.9532 - val_loss: 0.0115 - val_accuracy: 0.9545
Epoch 7/10
179/179 [==============================] - 226s 1s/step - loss: 0.0123 - accuracy: 0.9538 - val_loss: 0.0109 - val_accuracy: 0.9549

In [9]:
model5.compile(optimizer='Adam', loss='mean_squared_error', metrics = ['accuracy']) 
model5.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size1), steps_per_epoch=len(X_train)/batch_size1,
    epochs=20, verbose=1,validation_data=(X_val, y_val))

C:\Users\vip\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
89/89 [==============================] - 575s 5s/step - loss: 0.0852 - accuracy: 0.8682 - val_loss: 0.0646 - val_accuracy: 0.8604
Epoch 2/20
89/89 [==============================] - 232s 3s/step - loss: 0.0410 - accuracy: 0.9226 - val_loss: 0.0506 - val_accuracy: 0.9128
Epoch 3/20
89/89 [==============================] - 229s 3s/step - loss: 0.0281 - accuracy: 0.9405 - val_loss: 0.0374 - val_accuracy: 0.9321
Epoch 4/20
89/89 [==============================] - 225s 3s/step - loss: 0.0222 - accuracy: 0.9468 - val_loss: 0.0286 - val_accuracy: 0.9419
Epoch 5/20
89/89 [==============================] - 224s 2s/step - loss: 0.0195 - accuracy: 0.9492 - val_loss: 0.0262 - val_accuracy: 0.9442
Epoch 6/20
89/89 [==============================] - 226s 3s/step - loss: 0.0180 - accuracy: 0.9504 - val_loss: 0.0236 - val_accuracy: 0.9464
Epoch 7/20
89/89 [==============================] - 229s 3s/step - loss: 0.0170 - accuracy: 0.9513 - val_loss: 0.0227 - val_accuracy: 0.9469
Epoch 8/20
89

## Save your model

In [15]:
# Save model architecture and weights
from tensorflow import keras
model.trainable = False
model.compile(optimizer=optimizer, loss="mean_squared_error",metrics =['accuracy'])
model.save('full_CNN_model.h5')

# Show summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 80, 160, 3)        12        
_________________________________________________________________
conv2d (Conv2D)              (None, 78, 158, 2)        56        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 76, 156, 4)        76        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 38, 78, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 76, 4)         148       
_________________________________________________________________
dropout (Dropout)            (None, 36, 76, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 74, 4)         1

In [16]:
model1.trainable = False
model1.compile(optimizer=optimizer1, loss="mean_squared_error",metrics =['accuracy'])
model1.save('full_CNN_model1.h5')
# Show summary of model
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 80, 160, 3)        12        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 78, 158, 2)        56        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 76, 156, 4)        76        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 38, 78, 4)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 36, 76, 4)         148       
_________________________________________________________________
dropout_10 (Dropout)         (None, 36, 76, 4)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 74, 4)        

In [20]:
model2.trainable = False
model2.compile(optimizer=optimizer2, loss="mean_squared_error",metrics =['accuracy'])
model2.save('full_CNN_model2.h5')
# Show summary of model
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 80, 160, 3)        12        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 78, 158, 2)        56        
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 76, 156, 4)        76        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 38, 78, 4)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 36, 76, 4)         148       
_________________________________________________________________
dropout_20 (Dropout)         (None, 36, 76, 4)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 34, 74, 4)        

In [18]:
model3.trainable = False
model3.compile(optimizer=optimizer, loss="mean_squared_error",metrics =['accuracy'])
model3.save('full_CNN_model3.h5')
# Show summary of model
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 80, 160, 3)        12        
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 78, 158, 2)        56        
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 76, 156, 4)        76        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 38, 78, 4)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 36, 76, 4)         148       
_________________________________________________________________
dropout_30 (Dropout)         (None, 36, 76, 4)         0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 34, 74, 4)        

In [12]:
model4.trainable = False
model4.compile(optimizer='Adam', loss="mean_squared_error",metrics =['accuracy'])
model4.save('C:/Users/vip/Documents/CESS/deep learning/labs/project/Baseline/full_CNN_model4.h5')
# Show summary of model
model4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 80, 160, 3)        12        
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 78, 158, 2)        56        
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 76, 156, 4)        76        
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 38, 78, 4)         0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 36, 76, 4)         148       
_________________________________________________________________
dropout_40 (Dropout)         (None, 36, 76, 4)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 34, 74, 4)        

In [13]:
model5.trainable = False
model5.compile(optimizer='Adam', loss='mean_squared_error',metrics =['accuracy'])
model5.save('C:/Users/vip/Documents/CESS/deep learning/labs/project/Baseline/full_CNN_model5.h5')
# Show summary of model
model5.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_5 (Batch (None, 80, 160, 3)        12        
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 78, 158, 2)        56        
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 76, 156, 4)        76        
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 38, 78, 4)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 36, 76, 4)         148       
_________________________________________________________________
dropout_50 (Dropout)         (None, 36, 76, 4)         0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 34, 74, 4)        